In [ ]:
import xlrd
import xlsxwriter
from collections import Counter
from itertools import chain
from janome.tokenizer import Tokenizer
import re
import got
import numpy as np
import pandas as pd
import csv
import math
from fractions import Fraction
import matplotlib.pyplot as plt
from math import *
import random
from tqdm import tqdm
import time
import itertools
from vincenty import vincenty
import matplotlib.pyplot as plt
import collections
from multiprocessing import Pool

In [ ]:
with open('csv_name', encoding="utf-8") as f:
    book = csv.reader(f)
    header = next(book)
    sheet = [row for row in book]
    sheet = np.asarray(sheet)
    sheet2 = np.asarray(sheet).T

In [ ]:
kazu = list(collections.Counter(sheet2[1]).values())
k = np.sum(kazu)
print(k)
print(len(kazu))

In [ ]:
print(kazu.count(288))
for a in range(len(kazu)):
    if kazu[a]!=288:
        print(a)
        print(kazu[a])

In [ ]:
zikoku = [sheet2[3][i] for i in tqdm(range(k), position=0, leave=True)]

In [ ]:
times = [[y for y in range(k) if zikoku[i]==zikoku[y]] for i in tqdm(range(288))]

In [ ]:
kanst = np.repeat((np.random.randint(0, 100, len(kazu))<10).view(np.int8), kazu)
sheet2 = np.append(sheet2, [kanst], axis=0)

In [ ]:
def kyori1(a,b):
    return vincenty((float(sheet2[4][a]), float(sheet2[5][a])), (float(sheet2[4][b]), float(sheet2[5][b])))
def kyori2(a,b):
    return vincenty((float(sheet2[4][a+1]), float(sheet2[5][a+1])), (float(sheet2[4][b+1]), float(sheet2[5][b+1])))
def kyori3(a,b):
    return vincenty((float(sheet2[4][a+2]), float(sheet2[5][a+2])), (float(sheet2[4][b+2]), float(sheet2[5][b+2])))

In [ ]:
sums315 = [0]*48
for a in tqdm(range(288)):#5分毎の24時間分
    for b in range(len(times[a])-1):#同時刻にいる人のリスト
        if int(sheet2[1][int(times[a][b])])==int(sheet2[1][int(times[a][b])+1]):#連続したら
            if int(sheet2[10][int(times[a][b])])==2:#回復していたら
                sheet2[10][int(times[a][b])+1] = 2
            elif np.random.randint(0, 100)<1 and int(sheet2[10][int(times[a][b])])==1:#50パーセントで回復
                sheet2[10][int(times[a][b])+1] = 2
            elif int(sheet2[10][int(times[a][b])])==1:#感染中だったら
                sheet2[10][int(times[a][b])+1] = 1
        for c in range(b+1, len(times[a])):#接触判定用
            if math.floor(kyori1(int(times[a][b]), int(times[a][c]))*1000)<=3 and math.floor(kyori2(int(times[a][b]), int(times[a][c]))*1000)<=3 and math.floor(kyori3(int(times[a][b]), int(times[a][c]))*1000)<=3:#15分間3メートル以内なら
                sums315[math.floor(a/6)] += 1
                if int(sheet2[1][int(times[a][b])])==int(sheet2[1][int(times[a][b])+1]):#連続したら
                    if int(sheet2[10][int(times[a][b])])==0 and int(sheet2[10][int(times[a][c])])==1 and np.random.randint(0, 100)<50:#どちらかが感染でもう一方が未感染
                        sheet2[10][int(times[a][b])+1] = 1
                    elif int(sheet2[10][int(times[a][b])]==1) and int(sheet2[10][int(times[a][c])])==0 and np.random.randint(0, 100)<50:#どちらかが感染でもう一方が未感染
                        sheet2[10][int(times[a][c])+1] = 1


In [ ]:
koku=[i*0.5 for i in range(48)]
fig = plt.figure()
plt.plot(koku,sums315)
plt.title("30分毎の接触人数(3mで15分以上を接触とする)",fontname="MS Gothic")
plt.xlabel("時刻[時]",fontname="MS Gothic")
plt.ylabel("接触人数[人]",fontname="MS Gothic")
plt.xticks( np.arange(0, 25, 1))
plt.yticks( np.arange(0, 6000, 500))
plt.grid(True)
plt.show()
fig.savefig("plot1.png")

In [ ]:
sheet4 = [[sheet2[10][int(times[b*6][a])] for a in range(len(times[b*6]))] for b in range(48)]
kanjo0 = [sheet4[i].count('0') for i in range(48)]
kanjo1 = [sheet4[i].count('1') for i in range(48)]
kanjo2 = [sheet4[i].count('2') for i in range(48)]
fig = plt.figure()
plt.plot(koku, kanjo0, label="Uninfect")
plt.plot(koku, kanjo1, label="infect")
plt.plot(koku, kanjo2, label="Recover")
plt.title("30分毎の感染人数", fontname="MS Gothic")
plt.xlabel("時刻[時]", fontname="MS Gothic")
plt.ylabel("感染人数[人]", fontname="MS Gothic")
plt.xticks( np.arange(0, 24, 1))
plt.yticks( np.arange(0, 6000, 500))
plt.legend(loc=0) 
plt.grid(True)
plt.show() 
fig.savefig("plot2.png")

In [ ]:
fin = []
fin.append(sums315)
fin.append([sheet4[a].count('0') for a in range(48)])
fin.append([sheet4[a].count('1') for a in range(48)])
fin.append([sheet4[a].count('2') for a in range(48)])

In [ ]:
with pd.ExcelWriter('result2.xlsx') as writer:
    pd.DataFrame(fin).to_excel(writer, sheet_name='sheet1')